# Workshop 6: Google Analytics Revenue Determination Crucial Factors -Data



### Contents of this Kernel

1. Problem Statement  
2. Understand the Dataset 


## 1. Problem Statement 

In this exercise https://www.kaggle.com/c/ga-customer-revenue-prediction , the aim is to analyze a Google Merchandise Store (also known as GStore, where Google swag is sold) customer dataset to predict revenue per customer. The exercise here is an explosition of the data and look into the GA data about the personas of the visitors and the analysis might lead to a better use of marketing budgets for those companies who choose to use data analysis on top of GA data. 

This exercise is to introduce a tool help to extract the important factors determinate your business objectives given you have done the exercise in Workshop 4 enable to format the data for analysis.

### Questions:

#### Q1: Seek out insights of the personas of the visitors to form crucial insights
#### Q2: Compare different kinds of boosting algorithms to find out the feature importances
As the first step, lets load the required libraries.

In [1]:
import datetime as dt
import numpy as np 
import pandas as pd 
import json
from pandas.io.json import json_normalize
import seaborn as sns 
import matplotlib.pyplot as plt 
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
from plotly import tools
init_notebook_mode(connected=True)

If your PC does not have plotly, you need to find way to install plotly to call in init_notebook_mode, iplot, plotly.graph_objs and tools

## 2. Understanding the Dataset

The data is shared in csv format. The csv files contains some filed with json objects. The description about dataset fields is given  https://www.kaggle.com/c/ga-customer-revenue-prediction/data 



### 2.1 Dataset Preparation

Lets read the dataset in csv format and unwrap the json fields. Students can reference on https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.json_normalize.html 
and
https://www.kaggle.com/julian3833/1-quick-start-read-csv-and-flatten-json-fields

In [2]:
# read in data
df = pd.read_csv("test.csv", nrows=100000)
df.shape

(100000, 13)

In [3]:
# See the variables 
df.head()

,channelGrouping,customDimensions,date,device,fullVisitorId,geoNetwork,hits,socialEngagementType,totals,trafficSource,visitId,visitNumber,visitStartTime
0,Organic Search,"[{'index': '4', 'value': 'APAC'}]",20180511,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",7460955084541987166,"{""continent"": ""Asia"", ""subContinent"": ""Souther...","[{'hitNumber': '1', 'time': '0', 'hour': '21',...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""4"", ""pageviews"": ""3"",...","{""referralPath"": ""(not set)"", ""campaign"": ""(no...",1526099341,2,1526099341
1,Direct,"[{'index': '4', 'value': 'North America'}]",20180511,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",460252456180441002,"{""continent"": ""Americas"", ""subContinent"": ""Nor...","[{'hitNumber': '1', 'time': '0', 'hour': '11',...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""4"", ""pageviews"": ""3"",...","{""referralPath"": ""(not set)"", ""campaign"": ""(no...",1526064483,166,1526064483
2,Organic Search,"[{'index': '4', 'value': 'North America'}]",20180511,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",3461808543879602873,"{""continent"": ""Americas"", ""subContinent"": ""Nor...","[{'hitNumber': '1', 'time': '0', 'hour': '12',...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""4"", ""pageviews"": ""3"",...","{""referralPath"": ""(not set)"", ""campaign"": ""(no...",1526067157,2,1526067157
3,Direct,"[{'index': '4', 'value': 'North America'}]",20180511,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",975129477712150630,"{""continent"": ""Americas"", ""subContinent"": ""Nor...","[{'hitNumber': '1', 'time': '0', 'hour': '23',...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""5"", ""pageviews"": ""4"",...","{""referralPath"": ""(not set)"", ""campaign"": ""(no...",1526107551,4,1526107551
4,Organic Search,"[{'index': '4', 'value': 'North America'}]",20180511,"{""browser"": ""Internet Explorer"", ""browserVersi...",8381672768065729990,"{""continent"": ""Americas"", ""subContinent"": ""Nor...","[{'hitNumber': '1', 'time': '0', 'hour': '10',...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""5"", ""pageviews"": ""4"",...","{""referralPath"": ""(not set)"", ""campaign"": ""(no...",1526060254,1,1526060254


In [4]:
# See the variables 
df.dtypes

channelGrouping         object
customDimensions        object
date                     int64
device                  object
fullVisitorId           uint64
geoNetwork              object
hits                    object
socialEngagementType    object
totals                  object
trafficSource           object
visitId                  int64
visitNumber              int64
visitStartTime           int64
dtype: object

#### it is noted in below how servers record some steamless data over interact with clients in the JSON format


Only "device", "geoNetwork", "totals", "trafficSource" are JSON objects whereas "customDimensions" and "hits" are different data nested dictionary objects. One will notice that the data strcuture of series of "customDimensions" and "hits" are more complicated, in this exercise, they are not included in the analysis.
"hits" record all the pages footprints. 

#### Use pandas.read_csv(converters) 
Dict of functions for converting values in certain columns. Keys can either be integers or column labels.

In [5]:
def load_df(csv_path='test.csv', nrows=None):
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
    
    df = pd.read_csv(csv_path, 
                     converters={column: json.loads for column in JSON_COLUMNS}, 
                     dtype={'fullVisitorId': 'str'}, # Important!!
                     nrows=nrows)
    
    for column in JSON_COLUMNS:
        column_as_df = json_normalize(df[column])
        # "normalize" convert semi-structured JSON data into a flat table.
        column_as_df.columns = [f"{column}.{subcolumn}" for subcolumn in column_as_df.columns]
        # using "." rather than "_" for subcolumn
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
    print(f"Loaded {csv_path}. Shape: {df.shape}")
    return df

#### Run the whole data may take hours depending the computing efficiency of your machine, it is suggested to limit to data size of 100000

In [6]:
train = load_df("test.csv", 100000)

/Users/victorius/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: FutureWarning:

pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead



Loaded test.csv. Shape: (100000, 59)


In [7]:
train = train.drop("customDimensions", axis = 1)
train = train.drop("hits", axis = 1)

In [8]:
train.to_csv(r'train_GA.csv', index = False)